In [4]:
import numpy as np
from PIL import Image
from pathlib import Path
from skimage import io
import matplotlib.pyplot as plt
from skimage.io import imread
from tifffile import imwrite, imsave
from os.path import split, join

This script takes .tiff stacks and normalizes them pixelwise, then saves them as a new image.  It can also subtract a mask from an iamge using the mask_subtract function.  

In [2]:
#This function is not finished
def linear_background_subtract(spectra):
    """This function subtracts a linear background by taking averaging the first three points and the last three points of a one dimensional
    array and """
    start = np.mean(spectra[0:3])
    end = np.mean(spectra[-1:-3])
    slope = (end-start)/len(spectra)

    for point,i in enumerate(spectra):
        br = slope*(i)+start
        spectra[i] = spectra[i]-br

#This function masks an image by multiplying by a binary (0, 1) mask
def mask_subtract(data: str, mask: str, outfile: str):
    """takes a numpy array and multiplies it buy a mask.  Data and mask must be the same dimensions, and mask must be valued 1-0."""
    image = io.imread(data)
    print(image.shape, image.dtype)
    original_dtype = image.dtype
    mask = io.imread(str(mask))
    print(mask.shape, mask.dtype)
    #this should normalize the mask to 0-1.  
    mask = (mask-np.min(mask))/(np.max(mask)-np.min(mask))
    print(np.min(mask), np.max(mask))

    try:
        try:
            for slice in range(image.shape[0]):
                image[slice,:,:] = image[slice,:,:]*mask[:,:,0]
        except:
                for slice in range(image.shape[0]):
                    image[slice,:,:] = image[slice,:,:]*mask[:,:]
    except: 
        print("Mask/Image dimension mismatch, masking skipped")

    image = image.astype(original_dtype)
    print(image.dtype)
    imsave(outfile, image)
    return image

#This function goes through each pixel in a hyperspectral image and normalizes it (0,1)
#This function is essentially equivalent to sklearn.preprocessing.MinMaxScaler, but I didn't know it existed when I wrote this program
def norm_tiff_stack(filename: str, fileoutname: str, br_subtract=False):

    """Takes a .tiff stack and returns the stack after normalizing it pixelwise from 0-1.
        Reads a file (filename), and writes a new file (fileoutname)"""

    #Read the tif stack and form a flattened array with the shape [spectrum, pixel]
    image = io.imread(str(filename))
    print(image.shape)
    flattened = np.reshape(image,(image.shape[0],image.shape[1]*image.shape[2]))

    #normalize by the max value of the intensity image
    normalized_data = np.zeros(flattened.shape)
    mins = np.min(flattened, axis=0)
    maxs = np.max(flattened, axis=0)
    for pixel in range(flattened.shape[1]):
        #Account for bad scans that would result in divide by zero errors
        spectrumrange = (maxs[pixel]-mins[pixel])
        if spectrumrange <0.001: spectrumrange=1

        #normalize the data from [0,1]
        normalized = (flattened[:,pixel]-mins[pixel])/spectrumrange
        normalized_data[:,pixel] = normalized
    reshaped = np.reshape(normalized_data,(image.shape[0], image.shape[1],image.shape[2]))
    print(reshaped.shape)
    reshaped = reshaped*65534
    reshaped = reshaped.astype(np.uint32)
    print(np.max(reshaped),np.min(reshaped),np.mean(reshaped))
    print(reshaped.dtype)
    #imwrite(fileoutname, reshaped)
    imsave(fileoutname, reshaped)
    

    """Trying to figure out how to get tiffile to write tiffs in a way that actually is readable by mantis and my computer, I thought it might need to save as uint16 but I guess that didn't do it either"""
    # outdata = (reshaped*65535)
    # outdata = outdata.astype('int')
    # imwrite(fileoutname,outdata)

    return reshaped



In [ ]:
mask_subtract()

Everything below in this function is just using the above functions to save cropped and normalized images

In [6]:
cr_72 = "/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/XAS_30um_2CA_2AN_570-584eV_0.2eV_step_Cr2p/XAS_30_um_2CA_2AN_570-584eV_0.2eV_step_Cr2p.tif"
ni_72 = "/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/XAS_30um_2CA_2AN_848-865eV_0.2eV_step_Ni2p/XAS_30um_2CA_2AN_848-865eV_0.2eV_step_Ni2p_aligned.tif"
al_72 = "/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/XAS_30um_2CA_2AN_75-90eV_0.2eV_step_AL2p/XAS_30um_2CA_2AN_75-90eV_0.2eV_step_Al2p_aligned.tif"
ti_72 = "/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/XAS_30um_2CA_2AN_452-470eV_0.2eV_step_Ti2p/XAS_30um_2CA_2AN_452-470eV_0.2eV_step_Ti2p_aligned.tif"
fe_72 = "/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/XAS_30um_2CA_2AN_703-718eV_0.2eV_step_Fe2p/XAS_30um_2CA_2AN_703-718eV_0.2eV_step_Fe2p.tif"
edge_mask = "/Users/apple/Sync/Research/Alloy 72/20 px edge mask.tif"
outdir = '/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/'


mask_subtract(cr_72,edge_mask,join(outdir,split(cr_72)[1]))
mask_subtract(ni_72,edge_mask,join(outdir,split(ni_72)[1]))
mask_subtract(al_72,edge_mask,join(outdir,split(al_72)[1]))
mask_subtract(ti_72,edge_mask,join(outdir,split(ti_72)[1]))
mask_subtract(fe_72,edge_mask,join(outdir,split(fe_72)[1]))

(72, 1024, 1024) uint16
(1024, 1024) uint8
0.0 1.0
uint16
(87, 1024, 1024) uint16
(1024, 1024) uint8
0.0 1.0
uint16
(77, 1024, 1024) uint16
(1024, 1024) uint8
0.0 1.0
uint16
(91, 1024, 1024) uint16
(1024, 1024) uint8
0.0 1.0
uint16
(77, 1024, 1024) uint16
(1024, 1024) uint8
0.0 1.0
uint16


array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [4]:
#calculating normalized images for valence band
vb_60 = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/VB-alighned-tiff/VBEEM_3rd-ox-alighned.tif"
vb_20 = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/VB-alighned-tiff/VBEEM_2nd-ox-alighned-2.tif"
vb_pre = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/VB-alighned-tiff/VBPEEM_pre-ox-not_aligned.tif"

vb_60_norm = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/VB-alighned-tiff/normed_VBEEM_3rd-ox-alighned.tif"
vb_20_norm = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/VB-alighned-tiff/normed_VBEEM_2nd-ox-alighned-2.tif"
vb_pre_norm = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/VB-alighned-tiff/normed_VBPEEM_pre-ox-not_aligned.tif"

norm_tiff_stack(vb_60,vb_60_norm)
norm_tiff_stack(vb_20,vb_20_norm)
norm_tiff_stack(vb_pre,vb_pre_norm)

(26, 1024, 1024)
(26, 1024, 1024)
65534 0 30761.509193383732
uint32
(28, 1024, 1024)
(28, 1024, 1024)
65534 0 28283.756004367555
uint32
(26, 1024, 1024)
(26, 1024, 1024)
65534 0 35289.32481549336
uint32


array([[[21105, 21021,  4228, ...,     0,     0, 49805],
        [52426, 43224,  5370, ..., 27802, 12600, 45368],
        [11442, 13796, 33464, ..., 23405, 46564, 55292],
        ...,
        [40958, 37449, 32767, ..., 10667, 17096, 16382],
        [ 2518,     0,  5957, ..., 36155, 16384,     0],
        [16386, 44838, 46812, ...,  6241,  9668, 17148]],

       [[ 5554, 22257, 11626, ..., 53618, 23130,  7862],
        [26214, 22309, 13965, ..., 23831, 15123, 25202],
        [13522, 17245, 37648, ..., 16381,     0, 34813],
        ...,
        [14337,     0,  4095, ..., 13715,     0,  7281],
        [35287, 46115, 23830, ..., 20339, 27851, 15419],
        [    0, 20690, 49151, ...,     0,  5370, 14698]],

       [[15550, 18548, 15854, ..., 44679, 65534, 10485],
        [ 9830, 12548, 12892, ..., 29790,  7559,     0],
        [12483,  3449, 22309, ..., 14043, 29317, 47100],
        ...,
        [43008, 21842, 43008, ..., 15240, 24218, 38228],
        [30246, 38836, 65534, ..., 11300, 147

In [5]:
mask_subtract("median_plus_8000.tif","3rd_oxidation_mask.tif","median_8000_masked.tif")

(269, 1024, 1024) uint8
(1024, 1024) uint8
0.0 1.0
uint8


In [4]:
particlemask = '/Users/apple/Sync/Research/Drafts/XPEEM Lund paper/Figure Files/NNMA segmentation/particle segmented_NNMA_1.tif'
metalmask = '/Users/apple/Sync/Research/Drafts/XPEEM Lund paper/Figure Files/NNMA segmentation/metal segmented_NNMA_1.tif'
cr_45L = '/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/3rd_oxidation_45L/45L-adjusted-Cr XAS 570-595 step0p2.tif'
particles = mask_subtract(cr_45L,particlemask,'particle_xas.tif')
metal = mask_subtract(cr_45L, metalmask,'metal_xas.tif')

(101, 512, 512) uint16
(512, 512) uint8
0.0 1.0
uint16
(101, 512, 512) uint16
(512, 512) uint8
0.0 1.0
uint16


In [4]:
cr_45L = '/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/3rd_oxidation_45L/45L-adjusted-Cr XAS 570-595 step0p2.tif'
mask = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/1st_oxidation_5L/mask_512.png"



mask_subtract(cr_45L,mask,"test.tif")

# cr_45L = 'test.tif'
# newfile = '/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/3rd_oxidation_45L/normed_45L-Cr XAS 570-595 step0p2.tif'
# normalized_cr_45L = norm_tiff_stack(cr_45L,newfile)

uint16
(512, 512, 3) uint8
1.0 0.0
uint16


In [6]:
ni_45L = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/3rd_oxidation_45L/45L-adjusted-Ni2p XAS 845-875 step0p2-1.tif"
mask = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/1st_oxidation_5L/mask_1024.png"
newfile = '/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/3rd_oxidation_45L/normed_45L-adjusted-Ni2p XAS 845-875 step0p2-1.tif'

mask_subtract(ni_45L,mask,"temp.tif")
normalized_ni_45L = norm_tiff_stack("temp.tif",newfile)

uint16
(1024, 1024, 3) uint8
1.0 0.0
uint16
(151, 1024, 1024)
(151, 1024, 1024)
65534 0 7765.159948942677
uint32


In [3]:
ni_5 = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/1st_oxidation_5L/5L-Ni2p XAS 845-875 step0p2-1-adjusted.tif"
mask = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/1st_oxidation_5L/mask_1024.png"
newfile = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/1st_oxidation_5L/normed_5L-Ni2p XAS 845-875 step0p2-1-adjusted.tif"

mask_subtract(ni_5,mask,"temp.tif")
normalized_ni_5L = norm_tiff_stack("temp.tif",newfile)

uint16
(1024, 1024, 3) uint8
1.0 0.0
uint16
(151, 1024, 1024)
(151, 1024, 1024)
65534 0 8125.157998173442
uint32


In [3]:
ni_15 = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/2nd_oxidation _15L/15L-corrected-Ni2p XAS 845-875 step0p2-1_adjusted.tif"
mask = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/1st_oxidation_5L/mask_1024.png"
newfile = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/2nd_oxidation _15L/normed_15L-corrected-Ni2p XAS 845-875 step0p2-1_adjusted.tif"
mask_subtract(ni_15,mask,"temp.tif")
normalized_ni_15L = norm_tiff_stack("temp.tif",newfile)


cr_15 = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/2nd_oxidation _15L/15L-Cr XAS 570-595 step0p1667_adjusted.tif"
mask = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/1st_oxidation_5L/mask_1024.png"
newfile = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/2nd_oxidation _15L/normed_15L-Cr XAS 570-595 step0p1667_adjusted.tif"
mask_subtract(cr_15,mask,"temp.tif")
normalized_cr_15L = norm_tiff_stack("temp.tif",newfile)



uint16
(1024, 1024, 3) uint8
1.0 0.0
uint16
(151, 1024, 1024)
(151, 1024, 1024)
65534 0 7863.094500693264
uint32
uint16
(1024, 1024, 3) uint8
1.0 0.0
uint16
(151, 1024, 1024)
(151, 1024, 1024)
65534 0 13902.786222704199
uint32


In [8]:
vb_15 = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/2nd_oxidation _15L/15L-VBPEEM91eVto96p4@0p2eVstep-hn100eV.tif"
mask = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/1st_oxidation_5L/mask_1024.png"
newfile = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/2nd_oxidation _15L/normed_15L-VBPEEM91eVto96p4@0p2eVstep-hn100eV.tif"
mask_subtract(vb_15,mask,"temp.tif")
normalized_vb_15L = norm_tiff_stack("temp.tif",newfile)



uint16
(1024, 1024, 3) uint8
1.0 0.0
uint16
(28, 1024, 1024)
(28, 1024, 1024)
65534 0 14556.439660719463
uint32
uint16
(1024, 1024, 3) uint8
1.0 0.0
uint16
(10, 1024, 1024)
(10, 1024, 1024)
65534 0 15595.296198940277
uint32


FileNotFoundError: [Errno 2] No such file or directory: '/Users/apple/vscode/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/3rd_oxidation_45L/new_VB-PEEMfrom91to96eVstep0p2-2_17-26_adjusted.tif'

In [9]:
vb_45 = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/3rd_oxidation_45L/VB-PEEMfrom91to96eVstep0p2-2_17-26_adjusted.tif"
mask = "/Users/apple/Sync/Research/maxPEEM - Ni-22Cr/1st_oxidation_5L/mask_1024.png"
newfile = "test2.tif"
mask_subtract(vb_45,mask,"temp.tif")
normalized_vb_15L = norm_tiff_stack("temp.tif",newfile)


uint16
(1024, 1024, 3) uint8
1.0 0.0
uint16
(10, 1024, 1024)
(10, 1024, 1024)
65534 0 15595.296198940277
uint32
